<a href="https://colab.research.google.com/github/Pablo-Valde/Segmentacion/blob/main/laboratorio_segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'>Laboratorio de Segmentación</h1>

<h2>1. Importación de Librerías</h2>

Para simplificar el ejercicio, primero importaremos las librerías, sin necesidad de levantar un ambiente virtual e instalarlas en éste, gracias a las virtudes de Google Colaboratory como editor. Otra de las ventajas de  este intérprete de Python, es que funciona con el formato de celdas de los *Jupyter Notebooks*. Esto permite trabajar bajo el paradigma de lo que se denomina *Literate Programming*, pudiéndose hacer un claro énfasis en la estructura lógica del programa.

Para ejecutar la celda a continuación, bastará que usted la seleccione apretando sobre ella con el cursor, y luego apriete <code>shift+enter</code>




In [1]:
# Visualization Libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Operational Libraries
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h2>2. Lectura de la Tabla de Datos</h2>

Para ejercitar, utilizaremos una tabla de datos (en adelante <i>dataset</i>) recolectados a partir de la encuesta de segmentación realizada a la salida de los locales Bimarc Food Market. Los datos no han sido procesados, por lo que hay ciertos campos que se transformarán y otros tantos que enriqueceremos en el proceso de segmentación.

Es importante notar que hay una columna numérica de identificación de los sujetos, denominada "Identificador". No se confunda, esta variable es arbitraria para todos los efectos prácticos, y no debe ser considerada para el análisis, salvo que sea considerada como llave relacional. Si usted no se encuentra familiarizado con el lenguaje y las librerías, Pandas incluye siempre un índice que comúnmente es también numérico. No confunda el índice con la columna de identificación.

In [2]:
# Importing the prospects dataset using pandas from a local CSV file
file_name = 'base-personas-ix-epf-(formato-csv).csv'  # Replace with the actual file name
dataset = pd.read_csv(file_name, delimiter=';', on_bad_lines='skip')

file_path_excel = 'diccionario-de-variables-ix-epf(1).xlsx'
xls = pd.ExcelFile(file_path_excel)

# Display of last 5 rows
dataset.tail(5)

,macrozona,folio_v,folio,fe,estrato_muestreo,var_unit,cse,persona,n_linea,npersonas,...,ite03,ite04,ite05,ite06,ite07,ite08,ite09,ite10,ocupadas,no_ocupadas
44683,4,14960,14960-1,"160,8926146970770",38,978,2,3,3,3,...,2,-77,"-77,000000000","-77,0000000000",2,"-77,000",2,"-77,000",2,1
44684,4,14961,14961-1,"103,1978379921670",11,875,1,1,1,4,...,2,-77,"-77,000000000","-77,0000000000",2,"-77,000",2,"-77,000",1,2
44685,4,14961,14961-1,"103,1978379921670",11,875,1,2,2,4,...,2,-77,"-77,000000000","-77,0000000000",2,"-77,000",2,"-77,000",2,1
44686,4,14961,14961-1,"103,1978379921670",11,875,1,3,3,4,...,2,-77,"-77,000000000","-77,0000000000",2,"-77,000",2,"-77,000",1,2
44687,4,14961,14961-1,"103,1978379921670",11,875,1,4,4,4,...,-77,-77,"-77,000000000","-77,0000000000",-77,"-77,000",-77,"-77,000",-77,-77


In [41]:
# Mostrar información general del dataset
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958198 entries, 0 to 958197
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   macrozona              958198 non-null  int64  
 1   folio_v                958198 non-null  int64  
 2   folio                  958198 non-null  object 
 3   n_linea                958119 non-null  float64
 4   fe                     958198 non-null  object 
 5   estrato_muestreo       958198 non-null  int64  
 6   var_unit               958198 non-null  int64  
 7   id_gasto               958198 non-null  object 
 8   cantidad               953792 non-null  object 
 9   unidad_medida          958198 non-null  object 
 10  glosa_ccif             958117 non-null  object 
 11  ccif                   958198 non-null  object 
 12  d                      958117 non-null  float64
 13  g                      958117 non-null  float64
 14  c                      958117 non-nu

<h2>3. Transformación de los Datos</h2>

A continuación procederemos a transformar los datos, creando un dataset numérico a partir del que descargamos. Si bien es más costoso en memoria el uso de réplicas completas de los datasets utilizados, esta práctica es conveniente cuando se trabaja con Jupyter Notebooks que podrían ser ejecutados en desorden o múltiples veces.

In [42]:
# Copying our dataset to avoid future issues
dataset_numerico = dataset.copy(deep=True)

Tambien nos interesa filtrar de antemano la base para ciertos aspectos que son de nuestro interes, por ejemplo filtrar para el sector de gran santiago

In [47]:
filtro_geografico = dataset_numerico[dataset_numerico['macrozona'] == 'Gran Santiago']
filtro_geografico = dataset_numerico[dataset_numerico['glosa_establecimiento'] == 'SUPERMERCADOS']
filtered_dataset = filtro_geografico[~filtro_geografico.applymap(lambda x: x == -77).any(axis=1)]
filtered_dataset = filtro_geografico[~filtro_geografico.applymap(lambda x: x == -77.0).any(axis=1)]
dataset.tail(5)

/tmp/ipykernel_4814/3863168281.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  filtered_dataset = filtro_geografico[~filtro_geografico.applymap(lambda x: x == -77).any(axis=1)]
/tmp/ipykernel_4814/3863168281.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  filtered_dataset = filtro_geografico[~filtro_geografico.applymap(lambda x: x == -77.0).any(axis=1)]


,macrozona,folio_v,folio,n_linea,fe,estrato_muestreo,var_unit,id_gasto,cantidad,unidad_medida,...,d,g,c,sc,p,gasto,glosa_establecimiento,cod_establecimiento,cantidad_inicial,unidad_medida_inicial
958193,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-15,"1,10714285714286e+03",ML,...,1.0,2.0,5.0,1.0,1.0,"1,43928571428571e+03",SUPERMERCADOS,1,"500,000",CC
958194,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-7,"1,10714285714286e+03",ML,...,1.0,2.0,9.0,1.0,2.0,"3,29928571428571e+03",SUPERMERCADOS,1,"500,000",CC
958195,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-8,"1,10714285714286e+03",ML,...,1.0,2.0,9.0,1.0,2.0,"3,29928571428571e+03",SUPERMERCADOS,1,"500,000",CC
958196,4,9999,9999-1,-77.0,"130,6225757230620",11,2869,9999-1-1,"1,00000000000000e+00",COMPRA,...,2.0,5.0,1.0,1.0,1.0,"2,48000000000000e+02",FERIA ARTESANAL - FERIA DE LAS PULGAS,20,"-77,000",-77
958197,4,9999,9999-1,-77.0,"130,6225757230620",11,2869,9999-1-2,"1,00000000000000e+00",-77,...,11.0,1.0,1.0,1.0,13.0,"6,32653061224490e+03",RESTAURANTES Y BARES,4,"-77,000",-77


Recordemos que los datos recolectados en una encuesta pueden ser de cuatro tipos principalmente:

- Nominal: nombres (identificación y clasificación)
- Ordinal: orden (jerarquización, posición relativa)
- Intervalo: cuantificación (cero arbitrario)
- Escala: cuantificación (cero absoluto)

Es importante que usted identifique el tipo de cada variable, para que así le sea más sencillo transformar los datos en información valiosa para su posterior análisis. Para efectos de este ejemplo, dividiremos la transformación de los datos por tipo para mayor claridad, transformando aquellas variables de tipo texto en numéricas según corresponda.

<h3>3.1. Valores Nominales</h3>

Las variables nominales son las siguientes en este caso: CCIF IX EPF(ccif), Glosa teórica CCIF IX EPF (glosa_ccif)(Corresponde al producto), Glosa teórica de los establecimientos(glosa_establecimiento), Cantidad o número de bienes o servicios adquiridos reportada(cantidad_inicial) y Unidad de medida de los bienes o servicios adquiridos reportada(unidad_medida_inicial). A continuación, haremos las transformaciones necesarias para este tipo de variables.

In [48]:
label_encoder = LabelEncoder()

In [45]:
# Identificador doesn't require further transformations

# Ejemplo: Codificar la columna 'glosa_establecimiento'
# Mapeo de la variable 'ccif'
filtered_dataset['ccif_encoded'] = label_encoder.fit_transform(filtered_dataset['ccif'])

# Mapeo de la variable 'glosa_ccif'
filtered_dataset['glosa_ccif_encoded'] = label_encoder.fit_transform(filtered_dataset['glosa_ccif'])

# Mapeo de la variable 'glosa_establecimiento'
filtered_dataset['glosa_establecimiento_encoded'] = label_encoder.fit_transform(filtered_dataset['glosa_establecimiento'])

# Mapeo de la variable 'unidad_medida_inicial'
filtered_dataset['unidad_medida_inicial_encoded'] = label_encoder.fit_transform(filtered_dataset['unidad_medida_inicial'])


# As well as Nacionalidad, Comuna doesn't have any order yet, so we'll do the same
#mapper_city = {city: i for i, city in enumerate(dataset_numerico['Comuna'].unique())}
#dataset_numerico['Comuna'] = dataset_numerico['Comuna'].replace(mapper_city)

# Local follows the same pattern as the last two variables
#mapper_local = {local: i for i, local in enumerate(dataset_numerico['Local'].unique())}
#dataset_numerico['Local'] = dataset_numerico['Local'].replace(mapper_local)

<h3>3.2. Variables Ordinales</h3>

Las variables ordinales en este caso son: División CCIF(d), Grupo CCIF(g), Clase CCIF(c), Subclase CCIF(sc), Producto CCIF(p) y Código interno de establecimiento para la clasificación de puntos de compra(cod_establecimiento).

In [49]:
# The column Nivel Educacional is ordered by the analyst from lower to higher
mapper_productos = {'S': 0, 'ND': 1,
                    'SD': 2, 'D': 3}

dataset_numerico['c'] = dataset_numerico['c'].replace(mapper_productos)

# Mapeo de la variable 'd' (División CCIF)
filtered_dataset['d_encoded'] = label_encoder.fit_transform(filtered_dataset['d'])

# Mapeo de la variable 'g' (Grupo CCIF)
filtered_dataset['g_encoded'] = label_encoder.fit_transform(filtered_dataset['g'])

# La variable 'c' ya fue mapeada anteriormente

# Mapeo de la variable 'sc' (Subclase CCIF)
filtered_dataset['sc_encoded'] = label_encoder.fit_transform(filtered_dataset['sc'])

# Mapeo de la variable 'p' (Producto CCIF)
filtered_dataset['p_encoded'] = label_encoder.fit_transform(filtered_dataset['p'])

# Mapeo de la variable 'cod_establecimiento' (Código interno de establecimiento)
filtered_dataset['cod_establecimiento_encoded'] = label_encoder.fit_transform(filtered_dataset['cod_establecimiento'])


In [50]:
filtered_dataset.tail(5)

,macrozona,folio_v,folio,n_linea,fe,estrato_muestreo,var_unit,id_gasto,cantidad,unidad_medida,...,d,g,c,sc,p,gasto,glosa_establecimiento,cod_establecimiento,cantidad_inicial,unidad_medida_inicial
958191,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-34,"3,76428571428571e+02",GR,...,1.0,2.0,2.0,1.0,1.0,"6,70928571428571e+03",SUPERMERCADOS,1,"170,000",GR
958192,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-35,"7,93883036253761e+02",GR,...,1.0,2.0,3.0,1.0,2.0,"3,85285714285714e+03",SUPERMERCADOS,1,"1,000",KG
958193,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-15,"1,10714285714286e+03",ML,...,1.0,2.0,5.0,1.0,1.0,"1,43928571428571e+03",SUPERMERCADOS,1,"500,000",CC
958194,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-7,"1,10714285714286e+03",ML,...,1.0,2.0,9.0,1.0,2.0,"3,29928571428571e+03",SUPERMERCADOS,1,"500,000",CC
958195,4,9998,9998-1,1.0,"158,4609379966400",38,2972,9998-1-8,"1,10714285714286e+03",ML,...,1.0,2.0,9.0,1.0,2.0,"3,29928571428571e+03",SUPERMERCADOS,1,"500,000",CC


<h3>3.3. Variables de Intervalo</h3>

Comúnmente las encuestas contendrán preguntas de tipo Likert con alternativas que buscan evaluar la opinión o satisfacción de los clientes respecto a ciertos tópicos. Este es un caso típico de variables de intervalo, y la encuesta solicitada por el Gerente de Marketing de Bimarc no es la excepción. La última pregunta realizada busca justamente medir la satisfacción con la experiencia de compra, cuestión que se codificó en escala Likert bajo la columna de Experiencia.

In [ ]:
# Experiencia is a column which contains answers to a Likert scale
mapper_experience = {'Extremadamente insatisfecho': 1, 'Algo insatisfecho': 2,
                     'Ni satisfecho ni insatisfecho': 3, 'Algo satisfecho': 4,
                     'Extremadamente satisfecho': 5}

dataset_numerico['Experiencia'] = dataset_numerico['Experiencia'].replace(
    mapper_experience)

<h3>3.4. Variables de Escala</h3>

También de libro, las variables de escala suelen ser la edad y montos finitos como el consumo medido en pesos. En esta caso en particular, por representar la cantidad de días al mes que visitan el supermercado, se considerará también la frecuencia. Esta última variable es más discutible, pues su medición se puede (y suele) hacer de otras formas.

Convenientemente, como todas estas variables ya son numéricas y contínuas, no realizaremos ningún tipo de transformación a continuación.

In [ ]:
# The age contained in column Edad, as well as the frequency with which our
# customer visits the supermarket in column Frecuencia, and the amount
# registered in the receipt of that visit in Consumo are all already numeric
# values and don't require further transformations

A continuación vamos a normalizar ciertos valores de la tabla

In [38]:
# Normalizar las columnas 'gasto', 'cantidad', y 'cantidad_inicial'
filtered_dataset['gasto_normalizado'] = (filtered_dataset['gasto'] - filtered_dataset['gasto'].min()) / (filtered_dataset['gasto'].max() - filtered_dataset['gasto'].min())
filtered_dataset['cantidad_normalizada'] = (filtered_dataset['cantidad'] - filtered_dataset['cantidad'].min()) / (filtered_dataset['cantidad'].max() - filtered_dataset['cantidad'].min())
filtered_dataset['cantidad_inicial_normalizada'] = (filtered_dataset['cantidad_inicial'] - filtered_dataset['cantidad_inicial'].min()) / (filtered_dataset['cantidad_inicial'].max() - filtered_dataset['cantidad_inicial'].min())

# Mostrar las primeras filas con las nuevas columnas normalizadas
filtered_dataset.head(5)

,macrozona,folio_v,folio,n_linea,fe,estrato_muestreo,var_unit,id_gasto,cantidad,unidad_medida,...,cod_establecimiento,cantidad_inicial,unidad_medida_inicial,glosa_establecimiento_encoded,ccif_encoded,glosa_ccif_encoded,unidad_medida_inicial_encoded,gasto_normalizado,cantidad_normalizada,cantidad_inicial_normalizada


<h2>4. Enriquecimiento de los Datos</h2>

Tras transformar los datos obtenidos a través de la encuesta, tenemos siempre la posibilidad de enriquecer la tabla cruzando la información de ésta con fuentes de terceros. En este caso, el equipo de Inteligencia de Negocios decidió cruzar la información de la comuna para crear un proxy de los ingresos, utilizando una tabla de pobreza por comuna y ubicación de éstas en la Región Metropolitana de Santiago. A continuación, podemos observar este procedimiento.

<h3>4.1. Lectura y Transformación de la Tabla de Comunas</h3>

In [ ]:
#Importing the prospects dataset using pandas
sheet_name = 'comunas_ejemplo'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
dataset_comunas = pd.read_csv(url)

# Transform Pobreza into a numeric variable
dataset_comunas['Pobreza'] = dataset_comunas['Pobreza'].str.replace('%', '')
dataset_comunas['Pobreza'] = dataset_comunas['Pobreza'].str.replace(',', '.')
dataset_comunas['Pobreza'] = dataset_comunas['Pobreza'].astype(float) / 100

<h3>4.2. Enriquecimiento de la Tabla Principal</h3>

In [ ]:
# Copying dataset_numerico to dataset_enriquecido
dataset_enriquecido = dataset_numerico.copy(deep=True)

# Detect position of Comuna column
comuna_index = dataset_enriquecido.columns.get_loc('Comuna')

# Insert new columns after it with an income proxy
dataset_enriquecido.insert(loc=comuna_index + 1, # and a broader location
                           column='Pobreza de la Comuna', value=None)

dataset_enriquecido.insert(loc=comuna_index, column='Ubicación', value=None)

# Reverse mapper_city keys and values
mapper_city_inv = {v: k for k, v in mapper_city.items()}

# Categorize Comuna by Ubicación
for i in dataset_enriquecido.index:
    row = dataset_enriquecido.loc[i]
    row_comuna = mapper_city_inv[row['Comuna']]

    # Getting poverty and location from dataset_comunas
    row_poberty = dataset_comunas[dataset_comunas['Comuna'] == row_comuna]
    row_ubicacion = dataset_comunas[dataset_comunas['Comuna'] == row_comuna]

    # Change values of the new columns for this index number
    dataset_enriquecido.loc[i, 'Pobreza de la Comuna'] = row_poberty['Pobreza'].values[0]
    dataset_enriquecido.loc[i, 'Ubicación'] = row_ubicacion['Ubicación'].values[0].title()

# Transform Ubicación and Pobreza de la Comuna to numeric
mapper_ubicacion = {ubicacion: i for i, ubicacion in enumerate(dataset_enriquecido['Ubicación'].unique())}
dataset_enriquecido['Ubicación'] = dataset_enriquecido['Ubicación'].replace(mapper_ubicacion)

dataset_enriquecido['Pobreza de la Comuna'] = dataset_enriquecido['Pobreza de la Comuna'].astype(float)

Gracias al enriquecimento del dataset principal, ahora disponemos de un dataset enriquecido con la ubicación y pobreza porcentual de las comunas de la Región Metropolitana de Santiago. Es importante que usted lea y revise el código, pues varios de los métodos utilizados serán fundamentales para el desarrollo de la tarea asociada a este ejemplo.

<h2>5. Revisión Primaria de la Tabla</h2>

A continuación, revisaremos los tipos de variables obtenidos, corrigiendo aquellos que pudieran dar lugar a problemas. También presentaremos su estadística descriptiva y si acaso existen valores anómalos. Finalmente, estudiaremos la relación entre los pares de variables mediante una matriz de pares o pairplot, para ver variables tentativas para segmentar.

<h3>5.1. Tipos de Variables</h3>

Previo a la revisión de los tipos de variables, copiaremos también el dataset, de tal forma que se pueda hacer luego una auditoria forense de los datos. Como se explicó previamente, en casos en que las tablas son muy grandes esto podría ser contraproducente, pero por lo general es preferible en tablas pequeñas trabajar con copias, antes de modificar sus datos.

In [ ]:
dataset_corregido = dataset_enriquecido.copy(deep=True)
print(dataset_corregido.dtypes)

Observemos que todas las variables son de tipo numérico, salvo por el Nivel Educacional. Esto suele pasar al transformar los datos, y se recomienda transformar los valores, antes que tratar de debuggear el problema.

<h3>5.2. Corrección de Valores</h3>

En algunos casos, la siguiente celda podría contener excepciones para no incluir en la estadística descriptiva variables que por su naturaleza no puedan o deban ser interpretadas de forma numérica. En esta ocación, bastará transformar el tipo de la columna Nivel Educacional a numérico. La función para transformar los datos se dejó abierta, de tal forma que usted la pueda utilizar para la tarea.

In [ ]:
# Setting columns as numeric
cols = ['Nivel Educacional']
dataset_corregido[cols] = dataset_corregido[cols].apply(pd.to_numeric,
                                                        errors='coerce')
dataset_corregido.dtypes

Observemos que la variable fue transformada a números decimales y no enteros. Esto se debe a la función to_numeric de pandas, y es recomendable para transformar en NA cualquier elemento que no pueda ser transformado a número. Observemos a continuación la estadística descriptiva de nuestra muestra:



In [ ]:
dataset_corregido.describe()

Nuevamente, el recuento de observaciones para las variables cubre la totalidad de la muestra, salvo para el Nivel Educacional. Dada la magnitud del problema, y lo simple de la pregunta subyacente, procederemos a filtrar para revisar manualmente. Por intuición, y dada la naturaleza del muestreo, podemos esperar ver errores de codificación de las respuestas.

<h3>5.3. Corrección Manual de Errores</h3>

In [ ]:
dataset[dataset_corregido['Nivel Educacional'].isna()].tail(3)

Efectivamente, observamos un error de tipeo que impedía que el diccionario para mapear la variable operara sobre las respuestas. Existen alternativas automatizadas que se basan en la similitud de las palabras para corregir este tipo de problemas, pero por la cantidad y simplicidad del problema, simplemente utilizaremos comandos de pandas para ello.

In [ ]:
dataset_corregido.loc[dataset_corregido['Nivel Educacional'].isna(),
                      'Nivel Educacional'] = 0

Volveremos a revisar la estadística descriptiva, para validar si acaso los cambios surtieron efecto:

In [ ]:
dataset_corregido.describe()

Otro aspecto clave a considerar, es la heterogénea distribución de la media y la varianza de las variables observadas. Esto podría dar lugar a problemas más adelante y, por lo general, se recomienda normalizar los datos para facilitar su trabajo.

<h2>6. Visualización de los Datos</h2>

Para visualizar los datos, lo más que podemos hacer es hacerlo en tres dimensiones. Ya analizamos la estadística descriptiva, que podría ser interpretada como la visualización en una dimensión. A continuación, haremos lo propio para las dimensiones que faltan.

<h3>6.1. Visualización de dos Dimensiones</h3>

Primero, y mediante la función <code>pairplot</code> de la librería seaborn, observaremos la estructura de los datos y sus relaciones entre pares. Con ello, podemos determinar visualmente qué variables presentan un mayor potencial para generar grupos distintos entre sí.

Pese a la advertencia acerca de la magnitud de los valores que toman nuestras variables, en el caso del pairplot esto no afectará la interpretación, ya que la escala de los ejes controla por este problema. No se preocupe si la ejecución de esta celda toma varios minutos, es normal y sólo debe esperar.

In [ ]:
# Excluding spurious variables from our columns' dataset
cols = [col for col in dataset_corregido.columns if col != 'Identificador']

# Plot pairwise relationships in our dataset
pairwise_plot = sns.pairplot(dataset_corregido[cols])

La mayoría de los gráficos generados corresponde a nubes de puntos sin mayor interpretación posible. Otros tantos son gráficos de variables booleanas o con muy pocas alternativas como para detectar patrones claros.

Sin embargo, sí hay ciertas variables que al interactuar una con la otra se dividen en grupos o marcan tendencias. Ejemplos de ello son la edad con el consumo, la pobreza de la comuna con el consumo y la frecuencia de compra con el consumo. No tomaremos en cuenta la comuna, pues esta variable nos sirvió de llave para medir la pobreza, y por lo tanto nuestro análisis se centrará la variable que derivamos.

<h3>6.2. Visualización en Tres Dimensiones</h3>

Evidentemente, el consumo será una variable que analizaremos. La edad no es tan atractiva, pues reconocemos dos grandes grupos, pero dentro de cada uno la edad es indistinta. La pobreza y la frecuencia sí serán variables atractivas y, si graficamos, podremos observar que efectivamente se crean grupos dentro de la muestra.



In [ ]:
# Visualize up to three variables using a 3d scatter plot with plotly
fig = px.scatter_3d(dataset_corregido, x='Consumo', y='Pobreza de la Comuna',
                    z='Frecuencia')

fig.update_layout(scene=dict(xaxis=dict(range=[600000, 0])))
fig.update_layout(scene=dict(yaxis=dict(range=[0.35, 0])))

fig.show()

Con los análisis anteriormente realizados, y si etiquetamos y revisamos exhaustivamente cada uno de los grupos, podríamos dar con una buena segmentación a priori.

No obstante, esto sólo nos servirá para identificar el número de segmentos hasta las tres dimensiones. Para segmentos cuyas variables críticas sean más de tres, tendremos que utilizar métodos estadísticos más complejos, como los que veremos a continuación.

<h2>7. Algoritmos de Segmentación Jerárquica</h2>

Se denomina segmentación jerárquica a aquella cuyos grupos comparten una raíz común en la muestra, separándose gradualmente y en forma interconectada en diferentes ramas. A continuación, importaremos las librerías específicas para realizar estas operaciones.

In [ ]:
# Clustering Libraries
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

<h3>7.1. Normalización, Depuración y Dendogramas</h3>

Normalmente, este ejercicio se inicia con la construcción de una gráfica denominada dendograma. En este caso, aplicaremos el método de mínima varianza de Ward para minimizar la varianza entre los clústers.

In [ ]:
# Preparing our plot
plt.figure(figsize=(10, 7))
plt.title("Customers Dendrogram")
plt.ylabel('Euclidean distances')
plt.xlabel('Customers')

# Selecting Annual Income and Spending Scores by index
excluded_cols = ['Identificador']
filtered_cols = [col for col in dataset_corregido.columns
                 if col not in excluded_cols]

selected_data = dataset_corregido[filtered_cols].copy(deep=True) #[['Consumo', 'Frecuencia', 'Pobreza de la Comuna']]

clusters = sch.linkage(selected_data, method='ward', metric="euclidean")

# Aproximation to the number of clusters
plt.axhline(y = 500, color = 'r', linestyle = '-')

# Creating and showing our dendogram
sch.dendrogram(Z=clusters)
plt.show()

Para determinar la cantidad de clústers, observaremos las distancias euclidianas, buscando el punto en que ellas son más largas, para luego trazar una línea horizontal aproximadamente por la mitad, en lo que se denomina "el método de Thorndike". La cantidad de verticales que atraviese es considerada una buena aproximación a la cantidad de clústers que tendremos en nuestra muestra [(Thorndike, 1953)](https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.177.7605).

Dada la segmentación a priori, sabemos que contamos con al menos tres segmentos. Aplicando el método de Thorndike, tendríamos que concluir que en la muestra se observan dos segmentos. Sin embargo, y como mencionamos previamente, sabemos que en nuestra muestra hay variables cuya magnitud podría estar afectando nuestros análisis. Una forma de controlar por ello es normalizando los datos, cuestión que haremos a continuación.

In [ ]:
dataset_normalized = dataset_corregido.copy(deep=True)

# Normalize each column of the dataset
for column in dataset_normalized.columns:
    # Check if column is numeric
    if dataset_normalized[column].dtype == 'object':
        continue

    # Normalize column
    dataset_normalized[column] = (dataset_normalized[column] - dataset_normalized[column].min()) / \
                                  (dataset_normalized[column].max() - dataset_normalized[column].min())

    # Transform NA values to zero
    dataset_normalized[column] = dataset_normalized[column].fillna(0)

    # Multiply by 100 to get a percentage and transform to int
    dataset_normalized[column] = (dataset_normalized[column] * 100).astype(int)

Tenga en cuenta, que para ejecutar el algoritmo que genera el dendograma, fue necesario transformar los valores normalizados de números decimales a enteros, multiplicando por 100 y forzando las columnas a tipo entero. Así, habiendo normalizado y transformado los datos, podemos analizar nuevamente el dendograma y evaluar si se corrigieron o no los problemas.

In [ ]:
# Preparing our plot
plt.figure(figsize=(10, 7))
plt.title("Customers Dendrogram")
plt.ylabel('Euclidean distances')
plt.xlabel('Customers')

# Selecting Annual Income and Spending Scores by index
excluded_cols = ['Identificador']
filtered_cols = [col for col in dataset_normalized.columns
                 if col not in excluded_cols]

selected_data = dataset_normalized[filtered_cols].copy(deep=True)
clusters = sch.linkage(selected_data, method='ward', metric="euclidean")

# Creating and showing our dendogram
sch.dendrogram(Z=clusters)
plt.show()

Lamentablemente, aún cuando normalizamos los datos, el dendograma sigue sin arrojar un resultado correcto. Esta vez, el problema se debe a la presencia de variables espúreas, ya que no contribuyen más que sólo ruido. Así, y con el afán de depurar nuestra base, reduciremos el caso a las variables que presentan los patrones más notorios en nuestro pairplot:

In [ ]:
# Preparing our plot
plt.figure(figsize=(10, 7))
plt.title("Customers Dendrogram")
plt.ylabel('Euclidean distances')
plt.xlabel('Customers')

# Selecting Annual Income and Spending Scores by index
excluded_cols = ['Identificador', 'Edad', 'Nacionalidad', 'Comuna',
                 'Ubicación', 'Local', 'Segmento', 'Experiencia']

filtered_cols = [col for col in dataset_normalized.columns
                 if col not in excluded_cols]

selected_data = dataset_normalized[filtered_cols].copy(deep=True)

clusters = sch.linkage(selected_data, method='ward', metric="euclidean")

# Aproximation to the number of clusters
plt.axhline(y = 500, color = 'r', linestyle = '-')

# Creating and showing our dendogram
sch.dendrogram(Z=clusters)
plt.show()

<h3>7.2. Visualización de los Segmentos Creados</h3>

Existen dos algoritmos para clusterizar en forma jerárquica: los aglomerativos y los divisivos. En este ejercicio utilizaremos una estrategia aglomerativa, donde cada observación comienza en su propio grupo, y los pares de grupos son mezclados mientras uno sube en la jerarquía, en forma ascendente.

No obstante, y tanto para la determinación del número de grupos por medio del dendograma como para la clasificación de los individuos de nuestra muestra, se recomienda hacer uso de diferentes vínculos y métricas, y observar los resultados generados con una perspectiva de negocios [(Sampaio, 2022)](https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/).

In [ ]:
# Divisive Hierarchical Clustering using the euclidean distance and ward method
clustering_model = AgglomerativeClustering(n_clusters=3, metric='euclidean',
                                           linkage='ward')
clustering_model.fit(selected_data)

# Accessing vars of the clustering object
data_labels_v1 = clustering_model.labels_

# Visualization of our graph
sns.set(rc={'figure.figsize':(10, 7)})
sns.scatterplot(x='Consumo', y='Pobreza de la Comuna', data=selected_data,
                palette="rainbow", hue=data_labels_v1)\
                .set_title('Labeled Customer Data')

<h3>7.3. Asignación de los Segmentos a la Muestra</h3>

Por último, agregamos una columna a nuestro dataset, cuyo contenido viene dado por las operaciones previas y permite asignar el grupo correcto a cada individuo..

In [ ]:
# Copying our dataset to avoid future issues
dataset_segmented_v1 = dataset_corregido.copy(deep=True)

# Adding clusters and reverting values from numeric
segments = [f'cluster_{group + 1:02d}' for group in data_labels_v1]
dataset_segmented_v1.insert(loc=1, column='segment', value=segments)

dict_mappers = {'Sexo': mapper_sex, 'Nacionalidad': mapper_nationality,
                'Nivel Educacional': mapper_education,
                'Ubicación': mapper_ubicacion, 'Comuna': mapper_city,
                'Local': mapper_local, 'Experiencia': mapper_experience}

for col, mapper in dict_mappers.items():
  mapper_inv = {v: k for k, v in mapper.items()}
  dataset_segmented_v1[col] = dataset_segmented_v1[col].replace(mapper_inv)

dataset_segmented_v1.sort_values(by=['segment', 'Identificador'], inplace=True)
dataset_segmented_v1

<h2>8. Algoritmos de Segmentación No-Jerárquica</h2>

La segmentación no jerárquica implica la formación de grupos fusionando o dividiendo los clústers en forma no ramificada. Estas técnicas agrupan los datos para maximizar o minimizar algunos criterios de evaluación. K-Means es un algoritmo simple de *machine learning* de tipo no-supervisado que agrupa los datos en un número específico (k) de grupos.

In [ ]:
# Clustering Libraries
from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import kelbow_visualizer

<h3>8.1. Definición de la cantidad de Clústers</h3>

A continuación, filtraremos la mínima cantidad de variables con las que podemos correr el método del "codo" para definir la cantidad de segmentos a crear. Al igual que en el método jerárquico, seguramente necesitemos normalizar y omitir una serie de variables para que nuestro modelo sea parsimonioso.

De todas formas, es útil observar el proceso completo y ver cómo aplica en la práctica el concepto de que si entran datos de mala calidad a un modelo, los resultados de éste también serán de mala calidad. A este concepto se lo conoce por la sigla GIGO (Garbage In, Garbage Out) y se le atribuye a George Fuechsel (2004).

In [ ]:
# Selecting Annual Income and Spending Scores by index
excluded_cols = ['Identificador']
cols = [col for col in dataset_corregido.columns if col not in excluded_cols]
base_data = dataset_corregido[cols].copy(deep=True)

Filtrados ya los datos, crearemos una función que calcula el algoritmo de k-medias en un loop entre 2 y 11 clústers, para así determinar el óptimo observando la suma del cuadrado de las distancias entre grupos (WCSS, por sus siglas en inglés):

\begin{align}
WCSS = \sum_{i\in n} (X_i-Y_i)^2
\end{align}

A continuación, haremos el ejercicio manualmente en un *for loop*:

In [ ]:
def manual_elbow(selected_data):
  # Storing processed information and models
  df_kmeans = pd.DataFrame(columns=['clusters', 'wcss'])
  dict_kmeans = {}

  for i in range(1, 11):
    df_kmeans.loc[i, 'clusters'] = i

    # Definition of our k-means model and fitting our model
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300,
                    n_init=10, random_state=0)

    y_kmeans = kmeans.fit(selected_data)

    # Sum of squared distances to the closest cluster center
    df_kmeans.loc[i, 'wcss'] = kmeans.inertia_

    # Storing all variables for our specific
    dict_kmeans[i] = y_kmeans

  return {'df': df_kmeans, 'dict': dict_kmeans}

El número óptimo de clústers se define visualmente, observando el punto donde la pendiente de la curva se suaviza. El nombre del método viene dado imaginando que la curva fuera un brazo apoyado sobre una superficie, donde el punto de inflexión sería el codo.

In [ ]:
kmeans_base = manual_elbow(base_data)

# Plot the elbow graph
plt.plot(kmeans_base['df'].index.to_list(), kmeans_base['df']['wcss'].to_list())
plt.title('The Elbow Method Graph')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

# Visual proximation to the number of clusters
plt.axvline(x=2, color='r', linestyle='-')

# Plotting
plt.show()

Evidentemente, los datos son el problema y, en esencia, estamos en presencia de un problema similar al del punto 7.1. Conocida ya la solución, simplemente utilizaremos el DataFrame generado en ese punto, excluyendo también las mismas columnas.

In [ ]:
# Selecting Annual Income and Spending Scores by index
excluded_cols = ['Identificador', 'Edad', 'Nacionalidad', 'Comuna',
                 'Ubicación', 'Local', 'Segmento', 'Experiencia']

filtered_cols = [col for col in dataset_normalized.columns
                 if col not in excluded_cols]

selected_data = dataset_normalized[filtered_cols].copy(deep=True)
kmeans = manual_elbow(selected_data)

# Plot the elbow graph
plt.plot(kmeans['df'].index.to_list(), kmeans['df']['wcss'].to_list())
plt.title('The Elbow Method Graph')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

# Visual proximation to the number of clusters
plt.axvline(x=3, color='r', linestyle='-')

# Plotting
plt.show()

<h3>8.2. Cálculo Matemático del Óptimo</h3>

A continuación, se presenta una aproximación matemática al óptimo número de grupos a formarse en nuestro proceso de segmentación. Más información y detalle en el artículo de [Granville (2019)](https://www.datasciencecentral.com/how-to-automatically-determine-the-number-of-clusters-in-your-dat/).

In [ ]:
# Copying our dataset to avoid future issues
df_optimal = kmeans['df'].copy(deep=True)

for i in range(df_optimal.shape[0]):
  index = df_optimal.index[i]

  if i > 0: # Avoiding first value for which there's no delta
    delta01 = df_optimal.loc[index-1, 'wcss'] - df_optimal.loc[index, 'wcss']
    df_optimal.loc[index, 'Delta 1'] = delta01

  if i > 1: # Avoiding second value for which there's no delta
    delta02 = df_optimal.loc[index-1, 'Delta 1'] - df_optimal.loc[index, 'Delta 1']
    df_optimal.loc[index, 'Delta 2'] = delta02

    # Strenght calculations and filters
    strenght = df_optimal.loc[index-1, 'Delta 2'] - df_optimal.loc[index, 'Delta 1']
    df_optimal.loc[index, 'Strenght'] = next(s if s > 1 else None
                                             for s in [strenght])

df_optimal['Relative Strenght'] = df_optimal['Strenght'] / df_optimal.index
df_optimal.set_index(keys='clusters', drop=True, inplace=True)

# Optimal k by strenght values
max_strenght = df_optimal['Relative Strenght'].max()
optimal_k = df_optimal[df_optimal['Relative Strenght']==max_strenght].index[0]

# Display of calculation tables
display(df_optimal)

print(f'\nEl número óptimo es de {optimal_k} clústers.')

Lamentablemente, en este caso el resultado no es el esperado. Más que desalentar el uso de este método, considere este resultado como una motivación al uso de diversas aproximaciones al número correcto de clústeres. Esto explica la interrogante frecuente del por qué utilizar métodos no-jerárquicos, si ya concluimos con el modelo del punto 7, por ejemplo.

<h3>8.3. Definición Automática del Número de Clústers</h3>

Una de las bondades de utilizar Python frente a otras herramientas para hacer estadísticas, es el gran número de contribuidores que trabajan constantemente para simplificar los métodos y funciones más utilizados en sus respectivas áreas de estudio. Así, la librería <code>yellowbrick</code> nos ofrece una alternativa sencilla para la determinación del número de clústers mediante el método del codo.

Se recomienda utilizar este enfoque a futuro, aunque para el ejercicio propuesto se pide desarrollar el algoritmo mediante la estrategia anterior, pudiéndose corroborar los resultados con este enfoque.

In [ ]:
  # UMétodo rápido para determinar el número de clústers
kelbow_visualizer(KMeans(random_state=5, n_init='auto'),
                  selected_data, k=(1,11))

Gracias a yellowbrick, y por la coincidencia de los resultados de las aproximaciones a la cantidad de clústeres de ambos modelos, podemos tranquilamente afirmar que estamos en presencia de tres segmentos.

<h3>8.4. Visualización de los Segmentos Creados</h3>

Este paso no es realmente necesario, pero al igual que en el caso jerárquico, nos permite monitorear el rendimiento de los algoritmos aplicados en la clasificación realizada. Así, el objetivo es determinar visualmente si los clústers creados se condicen con la observación inicial y los resultados esperados.

In [ ]:
# Final clustering model
clustering_model = kmeans['dict'][3]

# Accessing vars of the clustering object
data_labels_v2 = clustering_model.labels_

# Visualization of our graph
sns.set(rc={'figure.figsize':(10, 7)})
sns.scatterplot(x='Consumo', y='Pobreza de la Comuna', data=selected_data,
                palette="rainbow", hue=data_labels_v2)\
                .set_title('Labeled Customer Data')

<h3>8.5. Asignación de los Segmentos a la Muestra</h3>

Por último, agregamos una columna a nuestro dataset, cuyo contenido viene dado por las operaciones previas y permite asignar el grupo correcto a cada individuo. Al contar con los DataFrames de uno y otro método, podremos determinar la efectividad de ambos modelos para reconocer cada segmento, además de analizar las diferencias entre ambos.

In [ ]:
# Copying our dataset to avoid future issues
dataset_segmented_v2 = dataset_corregido.copy(deep=True)

# Adding clusters and reverting values from numeric
segments = [f'cluster_{group + 1:02d}' for group in data_labels_v2]
dataset_segmented_v2.insert(loc=1, column='segment', value=segments)

dict_mappers = {'Sexo': mapper_sex, 'Nacionalidad': mapper_nationality,
                'Nivel Educacional': mapper_education,
                'Ubicación': mapper_ubicacion, 'Comuna': mapper_city,
                'Local': mapper_local, 'Experiencia': mapper_experience}

for col, mapper in dict_mappers.items():
  mapper_inv = {v: k for k, v in mapper.items()}
  dataset_segmented_v2[col] = dataset_segmented_v2[col].replace(mapper_inv)

dataset_segmented_v2.sort_values(by=['segment', 'Identificador'], inplace=True)
dataset_segmented_v2

<h2>9. Definición de Perfiles y Buyer Personas</h2>

Generar perfiles implica generar una tipología o descripción de los consumidores que conformarán el o los mercados objetivos seleccionados. Se basan en las características compartidas por las personas de un segmento y, cuando se expresan como grupo, al que se le asigna un nombre genérico, o en la “persona” de un individuo tipo que podemos inventar para efectos del ejercicio en cuestión, dan lugar al concepto de buyer persona. Idealmente, se debiera incluir una descripción, motivaciones, y el rol que la marca podría llevar a cabo en la forma de satisfacer sus necesidades.

<h3>9.1. Comparación de los Resultados</h3>

A continuación, revisaremos si se encontraron diferencias en los segmentos definidos por ambos modelos. Para esto, utilizaremos una función nativa de Pandas, con la que veremos las filas que difieren entre las tablas.

In [ ]:
# If needed, we adjust for the segments' number
mapper_segments = {'cluster_01': 'cluster_03', 'cluster_02': 'cluster_01',
                   'cluster_03': 'cluster_02'}

dataset_segmented_v2['segment'] = dataset_segmented_v2['segment'].replace(
    mapper_segments)

dataset_segmented_v2.sort_values(by=['segment', 'Identificador'], inplace=True)

df_diff = dataset_segmented_v1.compare(dataset_segmented_v2)
print(f'Se encontraron {df_diff.shape[0]} filas distintas entre las tablas...')

De encontrarse diferencias entre los modelos, y si son relativamente pocas, se recomienda filtrarlas y analizar luego de revisar la estadística descriptiva de cada segmento a cuál debieran pertenecer. Dado que en este caso ambas tablas son iguales, a continuación se definirá el dataset final tomando la primera tabla como referencia:

In [ ]:
# Copying our dataset to avoid future issues
dataset_segmented = dataset_corregido.copy(deep=True)

# Adding clusters and reverting values from numeric
segments = [f'cluster_{group + 1:02d}' for group in data_labels_v2]
dataset_segmented.insert(loc=1, column='segment', value=segments)

<h3>9.2. Estadística Descriptiva por Grupo</h3>

El siguiente bloque de código tiene por objetivo imprimir los promedio de cada columna para los distintos segmentos. Así, tras ejecutarlo veremos impresa esta información para cada grupo, pudiéndose construir fácilmente los perfiles a partir de esta información.

In [ ]:
# Selecting columns to describe
excluded_cols = ['Identificador', 'Segmento']

filtered_cols = [col for col in dataset_segmented.columns
                 if col not in excluded_cols]

df_clustered = dataset_segmented[filtered_cols].copy(deep=True)

# Setting main variables to iterate over
dict_clusters = {}
list_dicts = []

for group in df_clustered['segment'].unique():
  # Filtering values for descriptive analysis to be done
  df_group = df_clustered[df_clustered['segment']==group].copy(deep=True)

  # Filling our cluster dictionary with the descriptive analysis
  dict_clusters[group] = df_group.describe(percentiles=[])

for cluster, values in dict_clusters.items():
  local_dict = {}

  # Future cluster index aggregation and addition of dict to list
  local_dict['cluster'] = cluster

  # Adding the number of individuals of each cluster
  group = int(re.search('(\d)+', cluster).group(1))
  size = df_clustered[df_clustered['segment']==group].shape[0]
  local_dict['size'] = size

  # Adding all the other values
  local_dict = {**local_dict, **dict(values.loc['mean', :])}
  list_dicts.append(local_dict)

df_archetypes = pd.DataFrame(list_dicts)
df_archetypes.set_index(keys='cluster', drop=True, inplace=True)
df_archetypes.sort_index(ascending=True, inplace=True)
df_archetypes.index.name = None

pd.set_option('display.float_format', lambda x: '%.1f' % x)
display(df_archetypes)

Es mucho lo que podemos determinar a partir de una información tan básica como las medias para cada variable observada de los grupos. A continuación, analizaremos variable por variable los resultados:

*   Sexo: observamos que en el clúster 3 dominan hegemónicamente las mujeres, mientras en el resto de los grupos distribuye más cargado a los hombres.
*   Edad: no parece presentar una diferencia significativa entre grupos a primera vista. Habría que correr un test de diferencia de medias para validar.
*   Nacionalidad: esta no es una variable a la que hayamos prestado especial atención, dada la nube de puntos que formaba en el pairplot.
*   Nivel Educacional: es significativamente más bajo para el clúster 3 que para los demás, a primera vista.
*   Ubicación o Comuna: omitidos por servir como llave para nuestro proxy de ingresos.
*   Pobreza de la Comuna: esta es nuevamente una variable que para el tercer segmento se dispara.
*   Local: no amerita análisis aún, pero será importante cuando veamos el detalle nominal y geográfico.
*   Frecuencia: en este caso, es el clúster 2 el que presenta una frecuencia que se escapa de la media entre grupos.
*   Consumo: Otra vez es el clúster 2 el que destaca, señalando una visita más frecuente y para un consumo no doméstico.

Con estas nociones claras, podemos aventurar algunas hipótesis que validaremos contra los datos nominales y nuestro conocimiento circunstancial.

<h3>9.3. Propuesta de Perfiles</h3>

Los perfiles que se presentan a continuación nacen de un trabajo iterativo y acabado de estudio de las variables nominales que, junto con el conocimiento circunstancial y de mercados, permite al equipo de Inteligencia de Mercados de Bimarc proponer una explicación a cada segmento.

1.   Se identificaron tres clústeres de consumidores. El primero, conocido como "foodies", se caracteriza por compras esporádicas de bajo monto, alto nivel educativo y adquisitivo, y una pasión por la comida y la gastronomía. Les gusta explorar nuevos sabores y restaurantes, compartiendo sus experiencias en línea.
2.   El segundo clúster, llamado "chefs", realiza compras frecuentes y de alto monto, a menudo relacionadas con necesidades culinarias profesionales. Tienen un alto nivel educativo y buscan ingredientes de alta calidad para crear platos excepcionales.
3.   El tercer segmento, "asesoras del hogar", realiza compras que no coinciden con sus ingresos estimados y a menudo en áreas residenciales diferentes. Se cree que trabajan en hogares de alto poder adquisitivo, desempeñando un papel clave en la gestión de las necesidades culinarias y domésticas.

<h3>9.4. Creación de Buyer Personas</h3>

A continuación se presenta una propuesta de buyer personas para cada uno de los segmentos identificados. Tome en consideración que para ello fue necesario investigar las características de los perfiles propuestos, cuestión de la que usted debiera dar cuenta en su informe.

---
**Buyer Persona para "Foodies"**

Nombre: Ana<br>
Edad: 25-35 años.<br>
Nivel educativo: Ingeniera Comercial.<br>

Características clave:
*   Apasionada por la comida y la gastronomía.
*   Le encanta probar nuevos sabores y explorar diferentes cocinas.
*   Activa en redes sociales y blogs culinarios donde comparte sus experiencias.
*   Tiene un ingreso estable y dispone de un presupuesto moderado para comer fuera.

Necesidades y deseos:
*   Busca restaurantes de alta calidad y experiencias culinarias únicas.
*   Siempre está en busca de ingredientes gourmet para cocinar en casa.
*   Valora la calidad y autenticidad de los alimentos.

---
**Buyer Persona para "Chefs"**

Nombre: Carlos<br>
Edad: 35-50 años.<br>
Nivel educativo: Graduado en Escuela de Chef Profesional.<br>

Características clave:
*   Chef profesional o dueño de restaurante.
*   Experimentado en la cocina y apasionado por la gastronomía.
*   Conocedor de ingredientes y técnicas culinarias avanzadas.
*   Tiene un ingreso moderado y busca calidad sobre cantidad.

Necesidades y deseos:
*   Busca proveedores de alimentos de alta gama para su restaurante.
*   Siempre está en búsqueda de ingredientes frescos y de calidad.
*   Valora la innovación en la cocina y está dispuesto a invertir en ingredientes excepcionales.

---
**Buyer Persona para "Asesoras del Hogar"**

Nombre: María<br>
Edad: 45-55 años.<br>
Nivel educativo: Educación secundaria.<br>

Características clave:

*   Trabaja como asesora del hogar en una zona residencial de alto poder adquisitivo.
*   Responsable de la gestión de la cocina y compras de reposición del el hogar donde trabaja.
*   Tiene un ingreso moderado y sus compras personales no las realiza en los locales de Bimarc Food Market.

Necesidades y deseos:

*   Busca eficiencia en las compras y productos de calidad para el hogar.
*   Valora la conveniencia y la durabilidad de los productos.
*   Su enfoque principal es mantener el hogar bien abastecido y satisfacer las necesidades culinarias de los residentes.

---

Estos buyer personas permiten a los equipos de marketing un trabajo más natural, al conectar los segmentos con un perfil y luego una idea con la que el interlocutor del informe se puede familiarizar de forma más humana y comprensible.

<h2>10. Repuesta a las Preguntas del Caso</h2>

1.   ¿Cuál es el perfil demográfico de los clientes de Bimarc Food Markets?<br>
<b>Respuesta:</b> Analizar sexo, edad, nacionalidad, nivel educacional y comuna de recidencia por segmento.

2.   ¿Cuáles son los patrones de compra de los clientes en Bimarc Food?<br>
<b>Respuesta:</b> Analizar los montos y frecuencia por segmento.

3.   ¿Cómo explicaría usted el bajo monto promedio de las boletas?<br>
<b>Respuesta:</b> Existe sólo un segmento que representa realmente a los trabajadores de la industria de los alimentos, y estos compran con alta frecuencia. Lo anterior, diluye el monto de las boletas y crea el efecto observado por Ricardo Bimarc.

3.   En su opinión, ¿estaría canibalizando Bimarc Food Markets la clientela de los supermercados tradicionales de la compañía?<br>
<b>Respuesta:</b> Sí, pero sólo en el segmento de asesoras del hogar.
